In [1]:
import pandas as pd
import torch
import tqdm
import transformers

In [2]:
tokenizer = transformers.GPT2Tokenizer.from_pretrained("gpt2")
model = transformers.GPT2LMHeadModel.from_pretrained("gpt2_output/checkpoint-12000/")

In [3]:
input_ids = tokenizer.encode("Walter Hansel Cuvee Alyce Chardonnay 2015 [response]", return_tensors='pt')
greedy_output = model.generate(input_ids, max_length=100)
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Walter Hansel Cuvee Alyce Chardonnay 2015 [response]	"The 2015 Alyce Chardonnay is a blend of the best lots of Chardonnay from the estate vineyards. The wine is a blend of the best lots of Chardonnay from the estate vineyards. The wine is a blend of the best lots of Chardonnay from the estate vineyards.
The 2015 Alyce Chardonnay is a blend of the best


In [4]:
# activate beam search and early_stopping
beam_outputs = model.generate(
  input_ids, 
  max_length=100, 
  num_beams=5, 
  no_repeat_ngram_size=2,
  num_return_sequences=5, 
  early_stopping=True
)

print("Output:\n" + 80 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Output:
--------------------------------------------------------------------------------
0: Walter Hansel Cuvee Alyce Chardonnay 2015 [response]	The 2015 vintage of this wine was ranked #2 on the Wine Spectator's Top 10 Wines of 2016
<|startoftext|>	[prompt]aditionalChateau Ste. Michelle Cold Creek Vineyard Cabernet Sauvignon (1.5 Liter Magnum) 2013		"This is a big, bold wine with a lot going on in the nose and palate.
1: Walter Hansel Cuvee Alyce Chardonnay 2015 [response]	The 2015 vintage of this wine was ranked #2 on the Wine Spectator's Top 10 Wines of 2016
<|startoftext|>	[prompt]aditionalChateau Ste. Michelle Cold Creek Vineyard Cabernet Sauvignon (1.5 Liter Magnum) 2013		"This is a big, rich wine with a lot going on in the nose and palate.
2: Walter Hansel Cuvee Alyce Chardonnay 2015 [response]	The 2015 vintage of this wine was ranked #2 on the Wine Spectator's Top 10 Wines of 2016
<|startoftext|>	[prompt]aditionalChateau Ste. Michelle Cold Creek Vineyard Cabernet Sauvignon (1.5

In [5]:
greedy_output[0].shape

torch.Size([100])

In [4]:
# set seed to reproduce results. Feel free to change the seed though to get different results
#tf.random.set_seed(0)

input_ids = tokenizer.encode("Vendange Chardonnay [response]", return_tensors='pt')

# activate sampling and deactivate top_k by setting top_k sampling to 0
sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=150, 
    top_p=0.90,
    top_k=0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=False))

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Output:
----------------------------------------------------------------------------------------------------
Vendange Chardonnay [response]	"The 2002 vintage of the Sonoma Chardonnay from our friends at Laguna Vista Vineyards produced a very special wine, full of finesse, depth, and elegance. The bouquet has aromas of pear, kiwi, apple and white peach. The wine is full-bodied, crisp and elegant with a rich, mineral quality, balanced by ripe tropical fruit flavors, finishing with a long, crisp finish. Try this vintage pair this Chardonnay with spicy Asian dishes, smoked fish, roasted chicken, grilled vegetables and foie gras. 

"
<|startoftext|>	[prompt]	Maggio Family Vineyards


In [5]:
fake_names = pd.read_csv("data/fake/fake_names_13301_2020-05-20.csv")
print(fake_names.shape)

(13301, 2)


In [6]:
fake_names.head(3)

,Unnamed: 0,0
0,0,Lachos Cellars Sauvignon Blanc 2012
1,1,Dry Estated Bios de Bourting Sannero (375ML ha...
2,2,Tarodahadin Chaary Rivi 2010


In [8]:
response_prompt = "[response]"

generated_descriptions = {}
for fake_name in tqdm.tqdm(fake_names.iloc[:,1]):
  #print(fake_name)
  
  # Create token from fake wine name
  tokenized_name = tokenizer.encode(
    text=(fake_name + "\t" + response_prompt), 
    return_tensors='pt'
  )
  
  # Generate a fake description based on the name
  generated_description = model.generate(
    input_ids=tokenized_name,
    do_sample=True,
    max_length=150,
    top_p=0.90,
    top_k=0    
  )
  
  generated_descriptions[fake_name] = tokenizer.decode(
    token_ids=sample_output[0], 
    skip_special_tokens=False
  )

  0%|▎                                                                                                                | 34/13301 [02:34<16:57:12,  4.60s/it]Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


  1%|▌                                                                                                                | 69/13301 [05:12<16:27:31,  4.48s/it]Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


  1%|▉                                                                                                               | 104/13301 [07:44<15:10:34,  4.14s/it]Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


  1%|█▏                                                                                                              | 139/13301 [10:17<16:19:52,  4.47s/it]Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


  1%|█▍                                                                                                              | 174/13301 [12:52<16:13:30,  4.45s/it]Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


...

  7%|████████                                                                                                      | 979/13301 [1:12:11<14:47:55,  4.32s/it]Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


  8%|████████▎                                                                                                    | 1014/13301 [1:14:44<14:55:41,  4.37s/it]Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


  8%|████████▌                                                                                                    | 1049/13301 [1:17:18<14:59:30,  4.41s/it]Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


  8%|████████▋                                                                                                    | 1065/13301 [1:18:32<15:02:22,  4.42s/it]


KeyboardInterrupt: 

In [9]:
pd.DataFrame.from_dict(generated_descriptions.items())

,0,1
0,Lachos Cellars Sauvignon Blanc 2012,"Vendange Chardonnay [response]\t""The 2002 vint..."
1,Dry Estated Bios de Bourting Sannero (375ML ha...,"Vendange Chardonnay [response]\t""The 2002 vint..."
2,Tarodahadin Chaary Rivi 2010,"Vendange Chardonnay [response]\t""The 2002 vint..."
3,Le Valli Sis de Trach Sauvignon Blanc 2013,"Vendange Chardonnay [response]\t""The 2002 vint..."
4,Fotes Jadot Sauvignon Blanc 2005,"Vendange Chardonnay [response]\t""The 2002 vint..."
...,...,...
1060,Domaine de Bord Cote Russian 2015,"Vendange Chardonnay [response]\t""The 2002 vint..."
1061,Domain Fubers Thige Rouge (375ML half-bottle) ...,"Vendange Chardonnay [response]\t""The 2002 vint..."
1062,Dr. Loch Estate Chardonnay (375ML half-bottle)...,"Vendange Chardonnay [response]\t""The 2002 vint..."
1063,Jakubl Mays Blet 2016,"Vendange Chardonnay [response]\t""The 2002 vint..."


In [37]:
type(model)

transformers.modeling_gpt2.GPT2LMHeadModel

In [38]:
torch.cuda.is_available()

True